In [35]:
import pandas as pd 
import networkx as nx 

# Paths

In [36]:
base_data = "data/politician_data_inside"
cohort_data = "data/cohort/all_cohorts.csv"
pageviews_path = "data/pageviews/"
net_stats_path = "data/graphs_eff/"

# Load data

In [37]:
data = pd.read_pickle(base_data)

In [38]:
data.head()

,#DBpURL,ID,WikiURL,birthDate,deathDate,first_name,fpp_gender,full_name,gender,genderize,name,nationality,occupation,party
0,http://dbpedia.org/resource/Quincy_Timberlake,11,http://en.wikipedia.org/wiki/Quincy_Timberlake,1980-04-22 00:00:00,None,quincy,NaN,Quincy+Timberlake,male,NaN,"[ quincy timberlake , timberlake quincy ]",[kenyan],[politician],[]
5,http://dbpedia.org/resource/Nizamettin_Erkmen,144,http://en.wikipedia.org/wiki/Nizamettin_Erkmen,None,1990-10-24 00:00:00,nizamettin,NaN,Nizamettin+Erkmen,male,NaN,[ erkmen nizamettin ],[turkish],[politician],[]
6,http://dbpedia.org/resource/Claudio_Scajola,183,http://en.wikipedia.org/wiki/Claudio_Scajola,1948-01-15 00:00:00,None,claudio,NaN,Claudio+Scajola,male,NaN,[ claudio scajola ],[italian],[politician],[ forza italia (2013) ]
8,http://dbpedia.org/resource/Thomas_Clausen_(Lo...,246,http://en.wikipedia.org/wiki/Thomas_Clausen_(L...,1939-12-22 00:00:00,2002-02-20 00:00:00,thomas,NaN,Thomas+Clausen+(Louisiana),male,NaN,"[ thomas greenwood clausen , clausen thomas g...",[american],[politician],[ democratic party (united states) ]
9,http://dbpedia.org/resource/Yang_Ti-liang,248,http://en.wikipedia.org/wiki/Yang_Ti-liang,1929-06-30 00:00:00,None,yang,NaN,Yang+Ti-liang,male,NaN,[ ti-liang yang ],[],[politician],[]


In [39]:
cohort = pd.read_csv(cohort_data)

In [40]:
cohort.head()

,node,year
0,803946.0,2001
1,554382.0,2001
2,843335.0,2001
3,1143861.0,2001
4,81474.0,2001


# Add cohort data

In [41]:
def add_cohort_data(data, cohort):
    """ Merge cohort data with politician data
    """
    cohort_merged = data.merge(cohort, left_on=["ID"], right_on=["node"])
    cohort_merged = cohort_merged.drop(["node","fpp_gender","genderize"], axis=1)
    cohort_merged = cohort_merged.rename(index=str, columns={"year": "entered"})
    return cohort_merged

In [42]:
data.shape

(46479, 14)

In [43]:
cohort.shape

(46479, 2)

In [44]:
# cohort_merged = data.merge(cohort, left_on=["ID"], right_on=["node"])
# cohort_merged = cohort_merged.drop(["node","fpp_gender","genderize"], axis=1)

In [45]:
# cohort_merged = cohort_merged.rename(index=str, columns={"year": "entered"})
# cohort_merged.head()
# cohort_merged.columns

In [46]:
cohort_merged = add_cohort_data(data, cohort)

# Join cohorts with pageviews and statistics

In [47]:
pageviews = pd.read_csv(pageviews_path+"2016.csv")
pageviews.head()

,name_u,views
0,A._K._Antony,76364.0
1,A._T._M._Abdul_Mateen,3766.0
2,Abdul_Rahman_Mustafa,2096.0
3,Adam_Marshall,4105.0
4,Adnan_Kassar,3899.0


In [48]:
pageviews.shape

(46479, 2)

In [49]:
def graph_to_df(net_stats_path, year):
    """ Reads pcikeled graph and returns it as a dataframe
    """
    G = nx.read_gpickle("{}/{}_12_dir".format(net_stats_path, year))
    id_lst = [n for n,d in G.nodes_iter(data=True)]
    prop_lst = [d for n,d in G.nodes_iter(data=True)]
    ids = pd.DataFrame(id_lst)
    props = pd.DataFrame(prop_lst)
    data = ids.join(props).rename(columns={0:'id'})
    return data

In [50]:
# gdf = graph_to_df(net_stats_path, 2016)
# gdf.head()

In [51]:
# cols_to_use = gdf.columns.difference(cohort_merged.columns)
# cols_to_use

In [52]:
# mdf = pd.merge(cohort_merged, gdf[cols_to_use], left_on=["ID"], right_on=["id"])
# mdf.head()

In [53]:
# mdf["name_u"] = mdf.apply(lambda x: x["full_name"].replace("+", "_"), axis=1)

In [54]:
# pd.merge(mdf, pageviews, left_on=["name_u"], right_on=["name_u"])

In [55]:
# pageviews

In [56]:
def add_pw_stats(year, df, net_stats_path, pageviews_path):
    """ Joins network stats and pageviews with politician data
    """
    gdf = graph_to_df(net_stats_path, 2016)
    pageviews = pd.read_csv(pageviews_path+"{}.csv".format(year))
    stats = pd.read_pickle(net_stats_path+"{}_12_dir".format(year))
#     df = df["entered" == year]
    cols_to_use = gdf.columns.difference(df.columns)
    mdf = pd.merge(df, gdf[cols_to_use], left_on=["ID"], right_on=["id"])
    mdf["name_u"] = mdf.apply(lambda x: x["full_name"].replace("+", "_"), axis=1)
    df_final = pd.merge(mdf, pageviews, left_on=["name_u"], right_on=["name_u"])
    return df_final

# Save for specific years

In [57]:
save_path = "data/connected_sources/"

In [58]:
def save(df, path,year):
    """ Saves df as pcikle
    """
    path = path+str(year)
    df.to_pickle(path)
    print("File saved at {}".format(path))

In [59]:
# df = add_pw_stats(2008, cohort_merged, net_stats_path, pageviews_path)

In [60]:
# save(df, save_path, 2008)

In [61]:
# df = add_pw_stats(2016, cohort_merged, net_stats_path, pageviews_path)

In [62]:
# save(df, save_path, 2016)

In [63]:
# df = add_pw_stats(2009, cohort_merged, net_stats_path, pageviews_path)
# save(df, save_path, 2009)

In [64]:
# years = [2008,2009,2010,2011,2012,2013,2014,2016]
years = [2008,2009,2010,2012,2014,2016]

In [65]:
for year in years:
    df = add_pw_stats(year, cohort_merged, net_stats_path, pageviews_path)
    save(df, save_path, year)

File saved at data/connected_sources/2008
File saved at data/connected_sources/2009
File saved at data/connected_sources/2010
File saved at data/connected_sources/2012
File saved at data/connected_sources/2014
File saved at data/connected_sources/2016


In [ ]:
pd.read_csv()